<a href="https://colab.research.google.com/github/suna0107/ANN_DL101/blob/main/(250321)_2_2_HS_NAME_Representation_embedding_en_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[HS NAME Representation]**
- HS품명 → 384차원 임베딩 = [384,]

**[HS NAME 임베딩(영문)]**

(1) HS 계층정보를 포함한 임베딩 설계
- HS 코드 description에 hs 코드가 포함되어 있는 경우 -> 계층정보도 함께 임베딩 → 추후, HS 코드별 구분이 명확하게 될 수 있음

In [72]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# 데이터 로드 (예: 엑셀 파일 로드)
file_path = "/content/HS_CODE_PROCESSED_kr.xlsx"  # 파일 경로 설정
df= pd.read_excel(file_path)

# 1. HS 계층정보 필요한 HS코드만 계층 정보 추가

- 물품명에 계층정보 있을 때, "xx.xx, xxxx.xx"  가 들어가는 경우, 관련된 hs 코드 연결하기

## (1) 계층 정보를 포함하는 열의 수 확인

- 전체 총 16,715개행

- 계층정보 포함 행 : 401개(전체 중 2.4%)

- (원래 계획) 계층정보 포함 행 중 텍스트가 긴 것은 제외

-> (수정) 2.4% 면 계층정보 모두 반영하도록 진행


- 계층정보를 포함하는 행 필터링 -> 그 행의 단어 수 확인

In [73]:
df['영문']

,영문
0,"Live horses, asses, mules and hinnies.\n"
1,Horses :
2,Pure-bred breeding animals
3,For farm breeding
4,Other
...,...
16709,Other\n
16710,Other
16711,Ceramics\n
16712,Musical instruments\n


In [74]:
# 계층 정보 포함 행 필터링 : '영문' 열 내 'xx.xx' 또는 'xxxx.xx' 의 숫자를 포함하는 행 필터링
df[df["영문"].str.contains(r"\b\d{2}(\d{2})?\.\d{2}\b", regex=True, na=False)]

<ipython-input-74-294594169862>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df[df["영문"].str.contains(r"\b\d{2}(\d{2})?\.\d{2}\b", regex=True, na=False)]


,4,6,8,10,HS_CODE,제XX류,제XXXX.XX호,국문,영문,Referenced_HS,Referenced_HS_Description,SBERT_Input,Token_Count
161,207,0,0,0,207000000,제02류,제0207호,"제0105호의 가금(家禽)류의 육과 식용 설육(屑肉)(신선한 것, 냉장하거나 냉동한...","Meat and edible offal, of the poultry of headi...",['0105'],0105: 살아 있는 가금(家禽)류[닭(갈루스 도메스티쿠스(Gallus domest...,0207000000 제0105호의 가금(家禽)류의 육과 식용 설육(屑肉)(신선한 것...,96
317,302,0,0,0,302000000,제03류,제0302호,신선하거나 냉장한 어류[제0304호의 어류의 필레(fillet)와 그 밖의 어육은 ...,"Fish, fresh or chilled, excluding fish fillets...",['0304'],"0304: 어류의 필레(fillet)와 그 밖의 어육(잘게 썰었는지에 상관없으며, ...",0302000000 신선하거나 냉장한 어류[제0304호의 어류의 필레(fillet)...,87
318,302,1,0,0,302010000,제03류,제0302.01호,"연어과. 다만, 제0302.91호부터 제0302.99호까지의 식용 어류 설육(屑肉)...","Salmonidae, excluding edible fish offal of sub...","['0302.91', '0302.92', '0302.93', '0302.94', '...",0302.91: 간ㆍ어란(魚卵)ㆍ어백(魚白)\n 0302.92: 상어 지느러미\n ...,"0302010000 연어과. 다만, 제0302.91호부터 제0302.99호까지의 식...",70
323,302,2,0,0,302020000,제03류,제0302.02호,넙치류[플루로넥티대(Pleuronectidae)과ㆍ바디대(Bothidae)과ㆍ사이노...,"Flat fish (Pleuronectidae, Bothidae, Cynogloss...","['0302.91', '0302.92', '0302.93', '0302.94', '...",0302.91: 간ㆍ어란(魚卵)ㆍ어백(魚白)\n 0302.92: 상어 지느러미\n ...,0302020000 넙치류[플루로넥티대(Pleuronectidae)과ㆍ바디대(Bot...,153
329,302,3,0,0,302030000,제03류,제0302.03호,다랑어[터너스(Thunnus)속의 것으로 한정한다]ㆍ가다랑어(줄무늬 버니토)[카추워...,"Tunas (of the genus Thunnus), skipjack tuna (s...","['0302.91', '0302.92', '0302.93', '0302.94', '...",0302.91: 간ㆍ어란(魚卵)ㆍ어백(魚白)\n 0302.92: 상어 지느러미\n ...,0302030000 다랑어[터너스(Thunnus)속의 것으로 한정한다]ㆍ가다랑어(줄...,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16629,9615,0,0,0,9615000000,제96류,제9615호,빗ㆍ헤어슬라이드(hair-slide)와 이와 유사한 물품ㆍ머리핀ㆍ컬링핀(curlin...,"Combs, hair-slides and the like; hairpins, cur...",['8516'],"8516: 전기식의 즉시식ㆍ저장식 물가열기와 투입식 가열기, 난방기기와 토양가열기,...",9615000000 빗ㆍ헤어슬라이드(hair-slide)와 이와 유사한 물품ㆍ머리핀...,193
16657,9619,0,30,10,9619003010,제96류,NaN,"소호 제6108.21호, 제6108.22호, 제6108.29호, 제6111.20호,...","Of subheadings 6108.21, 6108.22, 6108.29, 6111...","['6108.21', '6108.22', '6108.29', '6111.20', '...",6108.21: 면으로 만든 것\n 6108.22: 인조섬유로 만든 것\n 6108...,"9619003010 소호 제6108.21호, 제6108.22호, 제6108.29호,...",194
16658,9619,0,30,20,9619003020,제96류,NaN,"제6113호, 제6208.91호, 제6208.92호, 제6208.99호의 것\n","Of heading 6113, of subheadings 6208.91, 6208....","['6113', '6208.91', '6208.92', '6208.99']",6113: 의류(제5903호ㆍ제5906호ㆍ제5907호에 해당하는 메리야스 편물이나 ...,"9619003020 제6113호, 제6208.91호, 제6208.92호, 제6208...",103
16664,9701,0,0,0,9701000000,제97류,제9701호,"회화ㆍ데생ㆍ파스텔(손으로 직접 그린 것으로 한정하며, 제4906호의 도안과 손으로 ...","Paintings, drawings and pastels, executed enti...",['4906'],4906: 설계도와 도안[건축용ㆍ공학용ㆍ공업용ㆍ상업용ㆍ지형학용이나 이와 유사한 용도...,"9701000000 회화ㆍ데생ㆍ파스텔(손으로 직접 그린 것으로 한정하며, 제4906...",142


## (2) 반영하는 방안

- '영문' 열 내 'xx.xx' 또는 'xxxx.xx'"  가 들어가는 경우, 관련된 hs코드 내용을 가져와서 임베딩 필요

### (step1) 데이터 전처리

- HS_CODE -> chapter, subheading 열 만들기

- HS_CODE에서 숫자 XX, XX.XX, XXXX.XX 따오기

(1) HS_CODE -> chapter, subheading 열 만들기

In [89]:
import numpy as np
# 항상 먼저 HS_CODE를 문자열 10자리로 정제
df["HS_CODE"] = df["HS_CODE"].astype(str).str.zfill(10)

# 1️⃣ Chapter 열 생성 (앞 2자리 → 두 자리 문자열로 저장)
df["Chapter"] = df["HS_CODE"].astype(str).str[:2]

# 2️⃣ Subheading 열 생성
def make_subheading(hs_code):
    hs_code = str(hs_code).zfill(10)

    if hs_code[6:] == "0000":
        if hs_code[4:6] == "00":
            # 5~6자리가 00 → 앞에서 4자리 → xx.xx 형식
            return f"{hs_code[:2]}.{hs_code[2:4]}"
        else:
            # 일반적인 xxxx.xx 형식
            return f"{hs_code[:4]}.{hs_code[4:6]}"

    return np.nan


df["Subheading"] = df["HS_CODE"].apply(make_subheading)


In [90]:
# 결과 확인 -> 국문열과 갯수 동일
df["Subheading"].isnull().sum()

np.int64(9121)

In [91]:
df["제XXXX.XX호"].isnull().sum()

np.int64(9121)

### (step2) "영문"내 참조 HS 있는 것, 없는 것 별로 임베딩 INPUT값 준비

1. 참조 HS 데이터 준비
(1) 영문 내 참조 HS 가져오기 => reference_hs_en 열 추가

(2) 추가 : XXXX to XXXX -> 참조 hs를 범위로 가져오도록 설정

2. 임베딩 INPUT 값 준비

(1) 참조해야하는 hs 가 없는 경우 :
HS CODE + XX + XXXX.XX + 영문

(2) 참조 hs가 있는 경우 :
HS CODE + 영문 + 참조 HS_DESCIPTION

(1) '영문'열에 hs 코드 df["Referenced_HS_EN"] 열 저장

In [93]:
import re

def extract_referenced_hs_en(text):
    if pd.isna(text):
        return []

    referenced = set()

    # 1️⃣ 범위 패턴: (xx.xx to xx.xx) 또는 (xxxx.xx to xxxx.xx)
    range_pattern = r"\b(\d{2,4}\.\d{2})\s*to\s*(\d{2,4}\.\d{2})\b"
    for start, end in re.findall(range_pattern, text):
        prefix_start, suffix_start = start.split(".")
        prefix_end, suffix_end = end.split(".")

        if prefix_start == prefix_end:
            # 같은 앞자리일 경우만 확장
            for i in range(int(suffix_start), int(suffix_end) + 1):
                referenced.add(f"{prefix_start}.{i:02d}")
        else:
            # 앞자리가 다르면 확장하지 않고 시작/끝만 포함
            referenced.add(start)
            referenced.add(end)

    # 2️⃣ 단일 코드 추출 (xx.xx 또는 xxxx.xx)
    single_pattern = r"\b\d{2,4}\.\d{2}\b"
    for match in re.findall(single_pattern, text):
        referenced.add(match)

    return sorted(referenced)


In [94]:
# 적용
df["Referenced_HS_EN"] = df["영문"].apply(extract_referenced_hs_en)

In [95]:
#결과 확인 : 리스트가 비어 있지 않은 행만 보기, 438개 행
df[df["Referenced_HS_EN"].apply(lambda x: isinstance(x, list) and len(x) > 0)][["영문", "Referenced_HS_EN"]].head(10)


,영문,Referenced_HS_EN
161,"Meat and edible offal, of the poultry of headi...",[01.05]
317,"Fish, fresh or chilled, excluding fish fillets...",[03.04]
318,"Salmonidae, excluding edible fish offal of sub...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
323,"Flat fish (Pleuronectidae, Bothidae, Cynogloss...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
329,"Tunas (of the genus Thunnus), skipjack tuna (s...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
339,"Herrings (Clupea harengus, Clupea pallasii), a...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
352,"Fish of the families Bregmacerotidae, Euclicht...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
360,"Tilapias (Oreochromis spp.), catfish (Pangasiu...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
366,"Other fish, excluding edible fish offal of sub...","[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ..."
389,"Fish, frozen, excluding fish fillets and other...",[03.04]


In [96]:
len(df[df["Referenced_HS_EN"].apply(lambda x: isinstance(x, list) and len(x) > 0)][["영문", "Referenced_HS_EN"]])

438

(2) 참조된 HS 코드의 설명 찾기

In [97]:
import re
import numpy as np

# HS_CODE → 영문 설명 딕셔너리
hs_en_dict = df.set_index("HS_CODE")["영문"].to_dict()

# ref는 그대로 출력하고, 매핑은 10자리 코드로
def describe_referenced_hs_en(reference_list):
    if not isinstance(reference_list, list) or len(reference_list) == 0:
        return np.nan

    descriptions = []

    for ref in reference_list:
        numeric = re.sub(r"[^\d]", "", ref)
        hs_10 = numeric.ljust(10, "0")

        if hs_10 in hs_en_dict:
            descriptions.append(f"{ref}: {hs_en_dict[hs_10]}")

    return descriptions if descriptions else np.nan




In [98]:

# 적용
df["Referenced_HS_Description_EN"] = df["Referenced_HS_EN"].apply(describe_referenced_hs_en)


In [99]:
# 결과확인
df[df["Referenced_HS_Description_EN"].notna()][["Referenced_HS_EN", "Referenced_HS_Description_EN"]].head(10)

,Referenced_HS_EN,Referenced_HS_Description_EN
161,[01.05],"[01.05: Live poultry, that is to say, fowls of..."
317,[03.04],[03.04: Fish fillets and other fish meat (whet...
318,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
323,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
329,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
339,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
352,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
360,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
366,"[0302.91, 0302.92, 0302.93, 0302.94, 0302.95, ...","[0302.91: Livers, roes and milt, 0302.92: Shar..."
389,[03.04],[03.04: Fish fillets and other fish meat (whet...


(3) SBERT 입력값 생성

In [100]:
def build_sbert_input_en(row):
    value = row["Referenced_HS_Description_EN"]

    # 설명이 있는 경우
    if isinstance(value, list) and len(value) > 0:
        ref_text = " ".join(value)
        return f"{row['HS_CODE']} {row['영문']} {ref_text}"

    # 설명이 없는 경우 → Subheading이 NaN이면 생략
    chapter = row.get("Chapter", "")
    subheading = row.get("Subheading", "")
    subheading_text = subheading if pd.notna(subheading) else ""

    return f"{row['HS_CODE']} {chapter} {subheading_text} {row['영문']}"

In [101]:
# 적용
df["SBERT_Input_EN"] = df.apply(build_sbert_input_en, axis=1)

In [102]:
# 결과
df[["HS_CODE", "SBERT_Input_EN"]].head()


,HS_CODE,SBERT_Input_EN
0,0101000000,"0101000000 01 01.01 Live horses, asses, mules ..."
1,0101020000,0101020000 01 0101.02 Horses :
2,0101210000,0101210000 01 0101.21 Pure-bred breeding animals
3,0101211000,0101211000 01 For farm breeding
4,0101219000,0101219000 01 Other


- 추가사항 : SBERT는 토큰수가 최대 512개 가능


In [103]:
from transformers import AutoTokenizer

# ✅ 사용 중인 SBERT 모델 이름
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

# ✅ 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(model_name)


# 7️⃣ 각 입력값의 토큰 개수 계산
df["Token_Count"] = df["SBERT_Input_EN"].apply(lambda x: len(tokenizer.tokenize(x)))

# 8️⃣ 512 토큰 초과하는 행 확인 : 2개
len(df[df["Token_Count"] > 512])

Token indices sequence length is longer than the specified maximum sequence length for this model (601 > 512). Running this sequence through the model will result in indexing errors


2

In [104]:
df[df["Token_Count"] > 512]

,4,6,8,10,HS_CODE,제XX류,제XXXX.XX호,국문,영문,Referenced_HS,Referenced_HS_Description,SBERT_Input,Token_Count,Chapter,Subheading,Referenced_HS_EN,Referenced_HS_Description_EN,SBERT_Input_EN
12874,8442,0,0,0,8442000000,제84류,제8442호,플레이트ㆍ실린더나 그 밖의 인쇄용 구성 부품의 조제용이나 제조용 기계류ㆍ장치ㆍ장비(...,"Machinery, apparatus and equipment (other than...","['8456', '8457', '8458', '8459', '8460', '8461...",8456: 각종 재료의 가공 공작기계[레이저나 그 밖의 광선ㆍ광자빔ㆍ초음파ㆍ방전ㆍ전...,8442000000 플레이트ㆍ실린더나 그 밖의 인쇄용 구성 부품의 조제용이나 제조용...,601,84,84.42,"[84.56, 84.57, 84.58, 84.59, 84.60, 84.61, 84....",[84.56: Machine-tools for working any material...,"8442000000 Machinery, apparatus and equipment ..."
13294,8466,0,0,0,8466000000,제84류,제8466호,제8456호부터 제8465호까지의 기계에 전용되거나 주로 사용되는 부분품과 부속품[...,Parts and accessories suitable for use solely ...,"['8456', '8457', '8458', '8459', '8460', '8461...",8456: 각종 재료의 가공 공작기계[레이저나 그 밖의 광선ㆍ광자빔ㆍ초음파ㆍ방전ㆍ전...,8466000000 제8456호부터 제8465호까지의 기계에 전용되거나 주로 사용되...,589,84,84.66,"[84.56, 84.57, 84.58, 84.59, 84.60, 84.61, 84....",[84.56: Machine-tools for working any material...,8466000000 Parts and accessories suitable for ...


(4) 최종 파일 저장 : '국문'열에 참조된 HS코드가 있는 경우 -> 그 내용 불러와서-> SBERT 입력 데이터 만든 파일

In [106]:
# 변환된 데이터 저장
output_path = "/content/HS_CODE_PROCESSED_kr_en_final.xlsx"
df.to_excel(output_path, index=False)

###(3) 전체 최종 코드

In [ ]:
import pandas as pd
import re

# 1️⃣ 데이터 로드
file_path = "/content/HS_CODE_PROCESSED_kr.xlsx"
df = pd.read_excel(file_path, dtype=str).fillna("")

import numpy as np
# 항상 먼저 HS_CODE를 문자열 10자리로 정제
df["HS_CODE"] = df["HS_CODE"].astype(str).str.zfill(10)

# 1️⃣ Chapter 열 생성 (앞 2자리 → 두 자리 문자열로 저장)
df["Chapter"] = df["HS_CODE"].astype(str).str[:2]

# 2️⃣ Subheading 열 생성
def make_subheading(hs_code):
    hs_code = str(hs_code).zfill(10)

    if hs_code[6:] == "0000":
        if hs_code[4:6] == "00":
            # 5~6자리가 00 → 앞에서 4자리 → xx.xx 형식
            return f"{hs_code[:2]}.{hs_code[2:4]}"
        else:
            # 일반적인 xxxx.xx 형식
            return f"{hs_code[:4]}.{hs_code[4:6]}"

    return np.nan

df["Subheading"] = df["HS_CODE"].apply(make_subheading)

# 4️⃣ 참조된 HS 코드의 설명 찾기 (현재 파일의 HS 코드 설명 매핑)

# HS_CODE → 영문 설명 딕셔너리
hs_en_dict = df.set_index("HS_CODE")["영문"].to_dict()

# ref는 그대로 출력하고, 매핑은 10자리 코드로
def describe_referenced_hs_en(reference_list):
    if not isinstance(reference_list, list) or len(reference_list) == 0:
        return np.nan

    descriptions = []

    for ref in reference_list:
        numeric = re.sub(r"[^\d]", "", ref)
        hs_10 = numeric.ljust(10, "0")

        if hs_10 in hs_en_dict:
            descriptions.append(f"{ref}: {hs_en_dict[hs_10]}")

    return descriptions if descriptions else np.nan

# 적용
df["Referenced_HS_Description_EN"] = df["Referenced_HS_EN"].apply(describe_referenced_hs_en)

# 6️⃣ SBERT 입력값 생성
def build_sbert_input_en(row):
    value = row["Referenced_HS_Description_EN"]

    # 설명이 있는 경우
    if isinstance(value, list) and len(value) > 0:
        ref_text = " ".join(value)
        return f"{row['HS_CODE']} {row['영문']} {ref_text}"

    # 설명이 없는 경우 → Subheading이 NaN이면 생략
    chapter = row.get("Chapter", "")
    subheading = row.get("Subheading", "")
    subheading_text = subheading if pd.notna(subheading) else ""

    return f"{row['HS_CODE']} {chapter} {subheading_text} {row['영문']}"

    # 적용
df["SBERT_Input_EN"] = df.apply(build_sbert_input_en, axis=1)

# 6️⃣ 최종 데이터 저장  -> 참조해야 하는 HS Description이 있는 경우 넣은 것
output_path = "/content/HS_CODE_PROCESSED_kr_en_final.xlsx"
df.to_excel(output_path, index=False)